In [1]:
import numpy as np
import astropy.units as u
from astropy.constants import c
from numpy.linalg import inv

In [2]:
H_0 = (70 * u.km/(u.s * u.Mpc))
c_H_0_2 = (c / (70 * u.km/(u.s*u.Mpc))**2).to(u.Mpc * u.s)

In [3]:
def chi(H, Om,OL,z):
    prefactor = 1 * (c / H_0).to(u.Mpc).value
    bracket = z + 0.5*z**2 * (-0.5*Om + OL - 1)
    return prefactor * bracket 

In [4]:
def dchi_H(H, Om,OL,z):
    prefactor =  -1 * c_H_0_2.value
    bracket =  z + 0.5*z**2 * (-0.5*Om + OL - 1)
    return prefactor * bracket

In [5]:
def dchi_dOm(H, Om,OL,z):
    return -1 * (c / (70 * u.km/u.s/u.Mpc)).to(u.Mpc).value * (0.25*z**2)

In [6]:
def dchi_dOL(H, Om,OL,z):
    return 1 * (c / (70 * u.km/u.s/u.Mpc)).to(u.Mpc).value * (0.5*z**2)

In [7]:
def construct_cov(H, Om,OL,z):
        
    return 0.01**2 * chi(H, Om,OL,z)**2 * np.identity(3)

In [8]:
def construct_mu_i(H, Om,OL,z):

    dch =  dchi_H(H, Om,OL,z) 
    dcm = dchi_dOm(H, Om,OL,z)
    dcl = dchi_dOL(H, Om,OL,z)

    mu_i = np.asarray([[dch,dch,dch],[dcm,dcm,dcm],[dcl,dcl,dcl]])
    
    return mu_i

In [9]:
def construct_mu_j(H, Om,OL,z):

    dch =  dchi_H(H, Om,OL,z) 
    dcm = dchi_dOm(H, Om,OL,z)
    dcl = dchi_dOL(H, Om,OL,z)

    mu_j = np.asarray([[dch,dcm,dcl],[dch,dcm,dcl],[dch,dcm,dcl]])
    
    return mu_j

In [10]:
def get_fisher_matrix(H, Om,OL,z):
    
    cov = construct_cov(H, Om,OL,z)
    mu_i = construct_mu_i(H, Om,OL,z)
    mu_j = construct_mu_j(H, Om,OL,z)
    
    return mu_i.T * inv(cov) * mu_j

In [11]:
F1 = get_fisher_matrix(H = H_0, Om = 0.3, OL = 0.7, z= 0.01)
F2 = get_fisher_matrix(H = H_0, Om = 0.3, OL = 0.7, z= 0.1)
F3 = get_fisher_matrix(H = H_0, Om = 0.3, OL = 0.7, z= 0.2)
F4 = get_fisher_matrix(H = H_0, Om = 0.3, OL = 0.7, z= 0.3)

Ftot = F1 + F2 + F3 + F4
np.sqrt(np.diag(inv(Ftot)))

array([1.13427275e-20, 1.00654615e-01, 5.03273076e-02])

In [12]:
(1.13427275e-20 * 1/u.s).to(u.km/u.s * 1/u.Mpc)

<Quantity 0.35 km / (Mpc s)>